<span style="color:blue">Thanks for using Drogon for your interactive Spark application. We update Drogon/SparkMagic as often as possible to make it easier, faster and more reliable for you. Have a question or feedback? Ping us on [uChat](https://uchat.uberinternal.com/uber/channels/spark).</span>

What's New
- Now you can use `%%configure` and `%%spark` magics to configure and start a Spark session (deprecating hard-to-use `%load_ext sparkmagic.magics` and `manage_spark` magics). Check out [this example](https://workbench.uberinternal.com/explore/knowledge/localfile/cwang/sparkmagic_python2_example.ipynb) for more details.
- Improved `%%configure` magic. You now can use it to make all Spark and Drogon configurations from within notebook itself. Check out our [latest documentation & examples](https://docs.google.com/document/d/1mkYtDHquh4FjqTeA0Fxii8lyV-P6qzmoABhmmRwm_00/edit#heading=h.xn14pmoorsn0) for more details.
- Bug fixes and performance updates.


### Spark Config

In [28]:
%%configure -f
{
  "pyFiles": [],
  "kind": "spark",
  "proxyUser": "radhesh",
  "sparkEnv": "SPARK_24",
  "driverMemory": "8g",
  "queue": "uber_eats_ml",
  "conf": { 
      "spark.dynamicAllocation.enabled": "true",
      "spark.dynamicAllocation.initialExecutors":100,
      "spark.dynamicAllocation.minExecutors":100,
      "spark.dynamicAllocation.maxExecutors" : 200,
      "spark.executor.memory": "12g",
      "spark.executor.memoryOverhead": "4g",
      "spark.driver.memory": "12g",
      "spark.driver.memoryOverhead" : "4g",
      "spark.hadoop.hadoop.security.authentication": "simple",
      "spark.shuffle.service.enabled" : true,
      "spark.sql.shuffle.partitions" : 500
},
  "executorCores": 2,
  "driverCores": 2,
  "executorMemory": "12g",
    "jars": ["/user/radhesh/spark-corenlp-0.4.0-spark2.4-scala2.11.jar",
            "/user/radhesh/stanford-corenlp-3.9.1-models.jar"],
   "drogonHeaders": {
    "X-DROGON-CLUSTER": "phx2/secure"
  }
}

Starting Spark application (can take 60s or more)...
Starting heartbeat thread...done.
Waiting for Drogon session to be ready............................................
Drogon session is ready.


Drogon Session ID,Spark Application ID,Kind,State,Spark UI,Driver log
514676579,application_1667252295751_3433208,spark,idle,,


SparkSession available as 'spark'.


Cell execution took 75 seconds.


In [2]:
spark

Starting Spark application (can take 60s or more)...
Starting heartbeat thread...done.
Waiting for Drogon session to be ready............................................
Drogon session is ready.


Drogon Session ID,Spark Application ID,Kind,State,Spark UI,Driver log
514020879,application_1667252295751_3207085,spark,idle,,


SparkSession available as 'spark'.


Cell execution took 75 seconds.
res1: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6ca8fe1

In [29]:
var input = spark.sql("SELECT * FROM tmp.grocery_items_tag_index")

input: org.apache.spark.sql.DataFrame = [tag: string, item_list: string]

In [30]:
input.show()

+------------------+--------------------+
|               tag|           item_list|
+------------------+--------------------+
|       chill drink|fanta fruit twist...|
|    latin american|ferrero duplo pie...|
|     cereal muesli|dorset cereal sim...|
|          homecare|anti bacterial de...|
|   craft brew beer|cabin brewing sup...|
|        rum import|blue chair bay ke...|
|goodee craving hou|haribo watermelon...|
|dairy product milk|darigold butter u...|
|  clean toiletries|durex extre safe ...|
|        sauce oils|lee kum kee soy s...|
|    aguas gaseosas|perrier lime , pe...|
| pastry individual|holiday cupcake l...|
|wash clean product|easy antibacteria...|
|     malter whisky|glenmorangie coll...|
|korean liquor wine|green grape korea...|
|  spirit wine beer|doom amber ale bo...|
|  snack parliament|mojito , arizona ...|
| alcohol multipack|heineken piece , ...|
|   traditional ale|guinness original...|
|     pasta lasagne|banetti fusillus ...|
+------------------+--------------

In [31]:
input.count()

res2: Long = 29091

In [40]:
input = input.withColumn("list",split(col("item_list"),","))

input: org.apache.spark.sql.DataFrame = [tag: string, item_list: string ... 2 more fields]

In [41]:
input.show()

+--------------------+--------------------+--------------------+--------------------+
|                 tag|           item_list|              number|                list|
+--------------------+--------------------+--------------------+--------------------+
|     wine white wine|harken california...|[harken californi...|[harken californi...|
|      frozen product|shime saba macker...|[shime saba macke...|[shime saba macke...|
|noodle instant pa...|noodle chicken po...|[noodle chicken p...|[noodle chicken p...|
|       france liquor|bol genever liquo...|[bol genever liqu...|[bol genever liqu...|
|     chocolate snack|rit bit cheese , ...|[rit bit cheese ,...|[rit bit cheese ,...|
| frozen meat seafood|catfish filet , n...|[catfish filet , ...|[catfish filet , ...|
|             ramadan|laziza biryanus y...|[laziza biryanus ...|[laziza biryanus ...|
|         logo burger|cheese burger wra...|[cheese burger wr...|[cheese burger wr...|
|produits exotique...|japanese kitkat b...|[japanese k

In [45]:
input = input.withColumn("size",size(col("list")))

input: org.apache.spark.sql.DataFrame = [tag: string, item_list: string ... 3 more fields]

### List of Tag - Item Index Where Items are >= 500

In [52]:
var largeItemList = input.select(col("tag"),col("item_list")).where($"size" >= 500)

largeItemList: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [tag: string, item_list: string]

In [53]:
largeItemList.count()

res16: Long = 913

In [54]:
largeItemList.show()

+--------------------+--------------------+
|                 tag|           item_list|
+--------------------+--------------------+
|                beer|boulevard rye rye...|
|champagne sparkli...|montelvinus asolo...|
|           crisp dip|kettle mature che...|
|             licence|dark horse chardo...|
|     hot dog sausage|polish kielbasa s...|
|             popcorn|newman touch butt...|
|           accessory|duramax suede lea...|
| sweet confectionery|haribo fizzy rain...|
|  confectionery mint|wrigley hubba bub...|
|      cough cold flu|ryukakusan herbal...|
|       cold sandwich|bologna cheese co...|
|     canadian whisky|canadian club pre...|
|         chill drink|fanta fruit twist...|
|      latin american|ferrero duplo pie...|
|            homecare|anti bacterial de...|
|   personal grooming|dive antiperspira...|
|            cleaning|cil bang black mo...|
|             produce|tomato red grape ...|
|               italy|caparzo brunello ...|
|           soda shop|sprecher c

### List of Tags which map to <= 4 items

In [55]:
var smallItemList = input.select(col("tag"),col("item_list")).where($"size" <= 4)

smallItemList: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [tag: string, item_list: string]

In [56]:
smallItemList.count()

res18: Long = 7124

In [57]:
smallItemList.show()

+--------------------+--------------------+
|                 tag|           item_list|
+--------------------+--------------------+
|     aquarium heater|halagen heater , ...|
|     complete cookie|peanut butter , p...|
|entree captain fried|wild gulf shrimp ...|
|         stick solid|mitchum advance c...|
|         escape game|exit dead oreint ...|
|    flavor ice cream|ice cream dip , d...|
|        bean haricot|selection haricot...|
|crisp snack scrat...|wild west origina...|
|bubble tea organi...|dirty chocolate b...|
|     barbotine slush|bubble slush slus...|
|noodle rice noodl...|ready yaki soba n...|
|        beer sleeman|sleeman honey bro...|
|fruit flavour pencil|blackcurrant penc...|
|tea coffee sugar ...|nescafe vanilla l...|
| yeastie boy brewery|digital ipa india...|
|         tinned meat|egeturk pinar sau...|
|           fruit pie|    apple lidded pie|
|        hey birthday|chocolate belgian...|
|  original milkshake|fudgsicle origina...|
|  sympathy sentiment|metal cros